<a href="https://colab.research.google.com/github/DragonMickey/DE_stepik/blob/main/14_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip --version

pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


In [2]:
!pip install pyspark py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=19fe6f84a998bc48235b16a544c9ca18bbd580253560a8217cf06c16e91ad7af
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, month
# Создание SparkSession
spark = SparkSession.builder.appName("Task 1. Weather").getOrCreate()

# Чтение CSV-файла
df = spark.read.csv("weather_data.csv", header=True, inferSchema=True)

# Проверка формата данных
df.printSchema()

# топ-5 самых жарких дней за все время наблюдений.
sorted_df1 = df.orderBy(col('temperature').desc()).select("date","temperature").limit(5)
sorted_df1.show()

# Найдите метеостанцию с наибольшим количеством осадков за последний год.
year = df.filter(year("date") == 2023)
sorted_df2 = year.groupBy("station_id").sum("precipitation").orderBy(col("sum(precipitation)").desc()).limit(1)
sorted_df2.show()

# Подсчитайте среднюю температуру по месяцам за все время наблюдений.
sorted_df3 = df.withColumn("month", month("date")).groupBy("month").avg("temperature").orderBy("month")
sorted_df3.show()

root
 |-- station_id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- temperature: double (nullable = true)
 |-- precipitation: double (nullable = true)
 |-- wind_speed: double (nullable = true)

+----------+------------------+
|      date|       temperature|
+----------+------------------+
|2021-08-20|39.982828249354846|
|2023-12-02| 39.96797489293784|
|2022-03-28|  39.8246894248997|
|2019-02-11| 39.76737697836647|
|2020-06-10| 39.69147838355929|
+----------+------------------+

+----------+------------------+
|station_id|sum(precipitation)|
+----------+------------------+
| station_5| 642.9302626767898|
+----------+------------------+

+-----+------------------+
|month|  avg(temperature)|
+-----+------------------+
|    1|11.356518462550754|
|    2| 9.067229891101926|
|    3| 7.244080205633994|
|    4|12.024529009744693|
|    5| 9.902883346912718|
|    6|13.421092297254138|
|    7|6.1857183016954576|
|    8|  10.9678002814186|
|    9| 9.596744236573942|
|   10|  9.098

In [18]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, count, year

spark = SparkSession.builder.appName("Task 2. Books and authors").getOrCreate()

books_df = spark.read.csv("books.csv", header=True, inferSchema=True)
authors_df = spark.read.csv("authors.csv", header=True, inferSchema=True)

books_df = books_df.withColumn("publish_date", to_date("publish_date", "YYYY-MM-DD"))
authors_df = authors_df.withColumn("birth_date", to_date("birth_date", "yyyy-MM-dd"))

books_authors_df = books_df.join(authors_df, on="author_id", how="left")
books_authors_df.show()
# Найдите топ-5 авторов, книги которых принесли наибольшую выручку.
top_authors = books_authors_df.groupBy("name").sum("price").orderBy(col("sum(price)").desc()).limit(5)
top_authors.show()

# Найдите количество книг в каждом жанре.
count_genre = books_authors_df.groupBy("genre").count().orderBy(col("count").desc())
count_genre.show()

# Подсчитайте среднюю цену книг по каждому автору.
avg_price_author = books_authors_df.groupBy("name").avg("price").orderBy(col("avg(price)").desc())
avg_price_author.show()

# Найдите книги, опубликованные после 2000 года, и отсортируйте их по цене.
books_after_2000 = books_authors_df.filter(year(col("publish_date")) > 2000).orderBy(col("price").desc())
books_after_2000.show()

+---------+-------+-------+-----------+-----+------------+---------+----------+---------+
|author_id|book_id|  title|      genre|price|publish_date|     name|birth_date|  country|
+---------+-------+-------+-----------+-----+------------+---------+----------+---------+
|        2|      1| Book_1|    Mystery|73.57|  1980-12-31| Author_2|1965-12-31|   Canada|
|        1|      2| Book_2|Non-Fiction| 41.1|  1982-12-31| Author_1|1960-12-31|    India|
|       10|      3| Book_3|    Fiction|10.63|  1984-12-31|Author_10|2005-12-31|    India|
|        9|      4| Book_4|Non-Fiction|46.31|  1986-12-31| Author_9|2000-12-31|Australia|
|        7|      5| Book_5|    Science|31.13|  1988-12-31| Author_7|1990-12-31|      USA|
|        4|      6| Book_6|Non-Fiction| 83.7|  1990-12-31| Author_4|1975-12-31|       UK|
|        6|      7| Book_7|Non-Fiction|40.36|  1992-12-31| Author_6|1985-12-31|      USA|
|        2|      8| Book_8|Non-Fiction|84.48|  1994-12-31| Author_2|1965-12-31|   Canada|
|        7

In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("Task 3. Movie and actors").getOrCreate()

actors_df = spark.read.csv("actors.csv", header=True, inferSchema=True)
movies_df = spark.read.csv("movies.csv", header=True, inferSchema=True)
movie_actors_df = spark.read.csv("movie_actors.csv", header=True, inferSchema=True)

# Регистрация DataFrame как временные таблицы
actors_df.createOrReplaceTempView("actors")
movies_df.createOrReplaceTempView("movies")
movie_actors_df.createOrReplaceTempView("movie_actor")

# Найдите топ-5 жанров по количеству фильмов
# top_genres = movies_df.groupBy("genre").count().orderBy(col("count").desc()).limit(5)
top_genres = spark.sql("""
SELECT genre, COUNT(*) count
FROM movies
GROUP BY genre
ORDER BY count DESC
LIMIT 5
""")
top_genres.show()

# Найдите актера с наибольшим количеством фильмов.
top_actor = spark.sql("""
SELECT a.name, COUNT(*) count
FROM actors a JOIN movie_actor ma USING(actor_id)
GROUP BY a.name
ORDER BY count DESC
LIMIT 1
""")
top_actor.show()

# Подсчитайте средний бюджет фильмов по жанрам.
avg_budget_genre = spark.sql("""
SELECT genre, AVG(budget) avg_budget
FROM movies
GROUP BY genre
ORDER BY avg_budget DESC
""")
avg_budget_genre.show()

# Найдите фильмы, в которых снялось более одного актера из одной страны.
movies_with_more1_actors_from_country = spark.sql("""
SELECT m.title, a.country, COUNT(*) count
FROM movies m JOIN movie_actor ma USING(movie_id) JOIN actors a USING(actor_id)
GROUP BY m.title, a.country
HAVING count > 1
""")
movies_with_more1_actors_from_country.show()

+------+-----+
| genre|count|
+------+-----+
| Drama|    6|
|Action|    6|
|Comedy|    4|
|Horror|    2|
|Sci-Fi|    2|
+------+-----+

+--------+-----+
|    name|count|
+--------+-----+
|Actor_17|    5|
+--------+-----+

+------+--------------------+
| genre|          avg_budget|
+------+--------------------+
|Horror|      8.7281876775E7|
|Sci-Fi|       7.809715175E7|
| Drama| 6.076021856166667E7|
|Comedy|     5.20709662225E7|
|Action|2.7492742561666667E7|
+------+--------------------+

+--------+---------+-----+
|   title|  country|count|
+--------+---------+-----+
| Movie_7|    India|    2|
| Movie_3|      USA|    2|
|Movie_10|       UK|    2|
|Movie_15|    India|    2|
|Movie_18|Australia|    2|
| Movie_1|    India|    3|
| Movie_2|      USA|    2|
| Movie_7|      USA|    2|
|Movie_10|      USA|    2|
+--------+---------+-----+

